In [ ]:
import boto3
import pandas as pd
import json
import psycopg2
import configparser

In [ ]:
config=configparser.ConfigParser()
config.read_file(open('cluster.config'))

In [ ]:
config.get('AWS','KEY')

In [ ]:

KEY=config.get('AWS','KEY')
SECRET=config.get('AWS','SECRET')
DWH_CLUSTER_TYPE=config.get('DWH','DWH_CLUSTER_TYPE')
DWH_NUM_NODES=config.get('DWH','DWH_NUM_NODES')
DWH_NODE_TYPE=config.get('DWH','DWH_NODE_TYPE')
DWH_CLUSTER_IDENTIFIER=config.get('DWH','DWH_CLUSTER_IDENTIFIER')
DWH_DB=config.get('DWH','DWH_DB')
DWH_DB_USER=config.get('DWH','DWH_DB_USER')
DWH_DB_PASSWORD=config.get('DWH','DWH_DB_PASSWORD')
DWH_PORT=config.get('DWH','DWH_PORT')
DWH_IAM_ROLE_NAME=config.get('DWH','DWH_IAM_ROLE_NAME')



In [ ]:
pd.DataFrame({"Param":
              ['DWH_CLUSTER_TYPE','DWH_NUM_NODES','DWH_NODE_TYPE','DWH_CLUSTER_IDENTIFIER','DWH_DB','DWH_DB_USER','DWH_DB_PASSWORD','DWH_PORT','DWH_IAM_ROLE_NAME'],
              "Value":
              [DWH_CLUSTER_TYPE,DWH_NUM_NODES,DWH_NODE_TYPE,DWH_CLUSTER_IDENTIFIER,DWH_DB,DWH_DB_USER,DWH_DB_PASSWORD,DWH_PORT,DWH_IAM_ROLE_NAME]
}
)

In [ ]:
ec2=boto3.resource('ec2',
                  region_name='ap-south-1',
                  aws_access_key_id=KEY,
                  aws_secret_access_key=SECRET)

In [ ]:
s3=boto3.resource('s3',
                 region_name='ap-south-1',
                  aws_access_key_id=KEY,
                  aws_secret_access_key=SECRET)


iam=boto3.client('iam',
                 region_name='ap-south-1',
                aws_access_key_id=KEY,
                aws_secret_access_key=SECRET
                )


redshift=boto3.client('redshift',
                      region_name='ap-south-1',
                  aws_access_key_id=KEY,
                  aws_secret_access_key=SECRET
)

In [ ]:
bucket=s3.Bucket("awsjupyterproject")
log_data_files=[filename.key for filename in bucket.objects.filter(Prefix='')]
log_data_files

In [ ]:
roleArn=iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']
roleArn

In [ ]:
try:
    response=redshift.create_cluster(
    ClusterType=DWH_CLUSTER_TYPE,
    NodeType=DWH_NODE_TYPE,
        
    DBName=DWH_DB,
    ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
    MasterUsername=DWH_DB_USER,
    MasterUserPassword=DWH_DB_PASSWORD,
    
    IamRoles=[roleArn]
    
    )
    
    
    
except Exception as e:
    print(e)

In [ ]:
redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]

In [ ]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth',-1)
    KeysToShow=['ClusterIdentifier','NodeType','ClusterStatus','MasterUsername','DBName',
 'Endpoint', 'VcpId']
    x=[(k,v) for k,v in props.items() if k in KeysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])


myClusterProps=redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

In [ ]:
DWH_ENDPOINT=myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN=myClusterProps['IamRoles'][0]['IamRoleArn']
DB_NAME=myClusterProps['DBName']
DB_USER=myClusterProps['MasterUsername']


In [ ]:
try:
    vpc=ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg=list(vpc.security_groups.all())[0]
    print(defaultSg)
    
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
    
except Exception as e:
    print(e)

In [ ]:
try:
    conn=psycopg2.connect(host=DWH_ENDPOINT, dbname=DB_NAME, user=DB_USER, password='', port=5439)
    
    
except psycopg2.Error as e:
    print("Error: Couldn'r make connection to postgresql database")
    print(e)

conn.set_session(autocommit=True)

In [ ]:
try:
    cur=conn.cursor()
except psycopg2.Error as e:
    print("Error: Could not get curser to the Database")
    print(e)

In [ ]:
try:
    cur.execute("""create table users(
    userid integer not null distkey sortkey,
    username char(8),
    firstname varchar(30),
    lastname varchar(30),
    city varchar(30),
    state char(10),
    email varchar(100),
    phone char(20),
    likesports boolean,
    liketheatre boolean,
    likeconcerts boolean,
    likejazz boolean,
    likeclassical boolean,
    likeopera boolean,
    likerock boolean,
    likevegas boolean,
    likebroadway boolean,
    likemusicals boolean);""")
    
except psycopg2.error as e:
    print("Error: Issue creating table")
    print(2)

In [ ]:
try:
    cur.execute("""
    copy users from 's3://awsjupyterproject/allusers_pipe.txt'
    credentials '<iam role>'
    delimiter '|'
    region 'ap-south-1'
    
    """)
    
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

In [ ]:
try:
    cur.execute("""
    select * from users;
    
    """)
    
except psycopg2.Error as e:
    print(e)

In [ ]:
row=cur.fetchone()
while row:
    print(row)
    row=cur.fetchone()

In [ ]:
redshift.delete_cluster(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER, SkipFinalClusterSnapshot=True)